# 영속적 메모리: 대화 기록 저장하기

이 노트북에서는 **MemorySaver**를 사용하여 **대화 기록을 저장하고 복원**하는 방법을 배웁니다.

## 영속성(Persistence)이란?

```
┌────────────────────────────────────────────────────────────────────┐
│               영속성 없음 vs 영속성 있음                           │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   영속성 없음 (이전 노트북):                                        │
│   • 프로그램 종료 → 대화 기록 사라짐 💨                            │
│   • 다시 시작하면 처음부터                                         │
│                                                                    │
│   영속성 있음 (이 노트북):                                          │
│   • 프로그램 종료 → 대화 기록 저장됨 💾                            │
│   • 다시 시작해도 이전 대화 기억                                   │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

## Thread ID란?

**Thread ID**는 대화를 구분하는 고유 식별자입니다.

```
┌────────────────────────────────────────────────────────────────────┐
│                    Thread ID의 역할                                │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   카카오톡 채팅방처럼 생각하세요!                                   │
│                                                                    │
│   Thread 1 (친구A와 대화):                                         │
│   ├─ "오늘 뭐해?"                                                  │
│   ├─ "영화 보러 가자"                                              │
│   └─ "좋아!"                                                       │
│                                                                    │
│   Thread 2 (친구B와 대화):                                         │
│   ├─ "숙제 했어?"                                                  │
│   └─ "아직..."                                                     │
│                                                                    │
│   → 각 Thread는 독립적인 대화 기록을 가짐                          │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

---

# 1. 환경 설정

In [ ]:
!pip install -q langchain langchain-ollama langgraph

In [ ]:
import subprocess
import time

!apt-get install -y zstd
!curl -fsSL https://ollama.com/install.sh | sh

subprocess.Popen(['ollama', 'serve'])
time.sleep(3)

!ollama pull llama3.2

# 2. 그래프 구성 (이전 노트북과 동일)

In [ ]:
from typing import Annotated, TypedDict
from langchain_core.messages import HumanMessage
from langchain_ollama import ChatOllama
from langgraph.graph import StateGraph, START, END, add_messages

# 상태 정의
class State(TypedDict):
    messages: Annotated[list, add_messages]

# 그래프 빌더
builder = StateGraph(State)

# LLM 모델
model = ChatOllama(model='llama3.2')

# 챗봇 노드
def chatbot(state: State):
    answer = model.invoke(state['messages'])
    return {'messages': [answer]}

# 노드와 엣지 추가
builder.add_node('chatbot', chatbot)
builder.add_edge(START, 'chatbot')
builder.add_edge('chatbot', END)

print("✅ 그래프 구성 완료")

# 3. MemorySaver로 영속성 추가

**MemorySaver**는 대화 상태를 메모리에 저장합니다.

```
┌────────────────────────────────────────────────────────────────────┐
│                    MemorySaver의 역할                              │
├────────────────────────────────────────────────────────────────────┤
│                                                                    │
│   그래프 실행 시:                                                  │
│   1. 입력 메시지 처리                                              │
│   2. 노드 실행                                                     │
│   3. 결과를 MemorySaver에 저장 ← 여기가 핵심!                      │
│                                                                    │
│   다음 실행 시:                                                    │
│   1. MemorySaver에서 이전 상태 불러오기                            │
│   2. 새 메시지와 합쳐서 처리                                       │
│   3. 결과 다시 저장                                                │
│                                                                    │
└────────────────────────────────────────────────────────────────────┘
```

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

# MemorySaver 생성
memory = MemorySaver()

# checkpointer 옵션과 함께 그래프 컴파일
graph = builder.compile(checkpointer=memory)

print("✅ 영속성이 추가된 그래프 컴파일 완료")

# 4. Thread ID를 사용한 대화

같은 Thread ID를 사용하면 이전 대화를 기억합니다.

In [ ]:
# Thread 설정 (대화방 ID라고 생각하세요)
thread1 = {'configurable': {'thread_id': '1'}}

print("=== Thread 1에서 대화 ===")

# 첫 번째 대화
print("\n[대화 1]")
print("사용자: 안녕하세요, 저는 민혁입니다!")
result1 = graph.invoke(
    {'messages': [HumanMessage(content='안녕하세요, 저는 민혁입니다!')]},
    thread1  # Thread ID 전달
)
print(f"AI: {result1['messages'][-1].content}")

In [ ]:
# 두 번째 대화 (같은 Thread)
print("\n[대화 2]")
print("사용자: 제 이름이 뭐죠?")
result2 = graph.invoke(
    {'messages': [HumanMessage(content='제 이름이 뭐죠?')]},
    thread1  # 같은 Thread ID
)
print(f"AI: {result2['messages'][-1].content}")

print("\n✅ 이전 대화를 기억합니다!")

# 5. 상태 확인하기

현재 저장된 대화 상태를 확인할 수 있습니다.

In [ ]:
# 현재 상태 확인
state = graph.get_state(thread1)

print("=== 현재 Thread 1의 상태 ===")
print(f"\n메시지 수: {len(state.values['messages'])}개\n")

for i, msg in enumerate(state.values['messages']):
    role = "사용자" if msg.type == "human" else "AI"
    content = msg.content[:50] + "..." if len(msg.content) > 50 else msg.content
    print(f"[{i+1}] {role}: {content}")

# 6. 상태 업데이트하기

대화 기록을 수동으로 수정할 수도 있습니다.

In [ ]:
# 상태 업데이트 (새 메시지 추가)
graph.update_state(
    thread1,
    {'messages': [HumanMessage(content='저는 LLM이 좋아요!')]}
)

print("✅ 상태 업데이트 완료")

# 업데이트된 상태 확인
state = graph.get_state(thread1)
print(f"\n업데이트 후 메시지 수: {len(state.values['messages'])}개")

# 7. 다른 Thread에서 대화

Thread ID가 다르면 완전히 독립적인 대화입니다.

In [ ]:
# 다른 Thread
thread2 = {'configurable': {'thread_id': '2'}}

print("=== Thread 2에서 대화 (새로운 대화) ===")

print("\n사용자: 제 이름이 뭐죠?")
result = graph.invoke(
    {'messages': [HumanMessage(content='제 이름이 뭐죠?')]},
    thread2  # 다른 Thread ID
)
print(f"AI: {result['messages'][-1].content}")

print("\n→ Thread 2는 Thread 1의 대화를 모릅니다!")

In [ ]:
# Thread 1로 돌아가면 여전히 기억함
print("=== 다시 Thread 1로 돌아가기 ===")

print("\n사용자: 제가 좋아하는 게 뭐라고 했죠?")
result = graph.invoke(
    {'messages': [HumanMessage(content='제가 좋아하는 게 뭐라고 했죠?')]},
    thread1  # 다시 Thread 1
)
print(f"AI: {result['messages'][-1].content}")

---

## 정리: 영속적 메모리

### 핵심 개념

| 개념 | 설명 | 비유 |
|------|------|------|
| **MemorySaver** | 상태를 저장하는 도구 | 저장 버튼 |
| **Thread ID** | 대화를 구분하는 ID | 채팅방 번호 |
| **checkpointer** | 저장 도구 연결 옵션 | 자동 저장 설정 |
| **get_state** | 저장된 상태 조회 | 기록 보기 |
| **update_state** | 상태 수동 수정 | 기록 편집 |

### 핵심 코드

```python
from langgraph.checkpoint.memory import MemorySaver

# 1. MemorySaver 생성
memory = MemorySaver()

# 2. checkpointer와 함께 컴파일
graph = builder.compile(checkpointer=memory)

# 3. Thread ID와 함께 실행
thread = {'configurable': {'thread_id': '1'}}
result = graph.invoke({'messages': [...]}, thread)

# 4. 상태 조회
state = graph.get_state(thread)

# 5. 상태 업데이트
graph.update_state(thread, {'messages': [...]})
```

### 주의사항

| 항목 | MemorySaver 특성 |
|------|------------------|
| **저장 위치** | 메모리 (RAM) |
| **프로그램 종료 시** | 데이터 사라짐 |
| **영구 저장 필요 시** | SQLite, PostgreSQL 등 사용 |

## 코드 변경점 (OpenAI → Ollama)

```python
# 원본
model = ChatOpenAI(model='gpt-4o-mini')

# 변경
model = ChatOllama(model='llama3.2')
```

## 다음 단계

대화가 길어질 때 **메시지를 자르는(Trim)** 방법을 배웁니다. (11번 노트북)